[Accueil](../../index.ipynb) > [Sommaire Première](../index.ipynb)

Ce TP est une application pratique du cours sur l'[algorithme des plus proches voisons](../8_Algorithmique/3_k_plus_proches_voisins.ipynb)

Ecrire une fonction *get_titanic_datas* qui récupère les données de l'url https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv et retourne une datframe de type panda.

Ourir ces données dans une tableur pour obseerver plus facilement les données

Observer les données et éliminer les colonnes qui selon vous n'ont pas d'importance sur la survie d'une passager.

- https://pixees.fr/informatiquelycee/n_site/nsi_prem_projet_titanic.html
- https://www.kaggle.com/competitions/titanic/data?select=test.csv
- https://medium.com/analytics-vidhya/titanic-machine-learning-by-k-nearest-neighbors-knn-algorithm-530d8bdd8323
- https://github.com/awesomedata/awesome-public-datasets/issues/351